# Introduction


Nous disposons plusieurs modèles pour traiter les données séquentielles. Ce type de modèle permet d'effectuer des tâches comme :
* Traduction automatique qui traduit une séquence d'une langue donnée dans une autre
* Sous-titrage d'images
* Résumé de textes
* ..

Plusieurs types d'architecture de réseaux ont été développés pour répondre à ces problématiques :
* Les réseaux récurrents (RNN)
* Long Short-Term Memory (LSTM)
* Gated Recurent Unit (GRU)
* ...

Ils sont essentiellement composés de deux parties
* Encodeur : fournit une représentation vectorielle de la séquence
* Décodeur : retourne une séquence à partir de la représentation vectorielle reçue de l'encodeur

Ces modèles ont des points faibles :
* Propagation et amplification des erreurs
* Explosion ou contraction du gradient
* ...

Les chercheurs de Google ont proposé les `Transformers` pour corriger les faiblesses de ces réseaux. C'est un modèle basé sur le [mécanisme d'attention](python-nlp-mecanisme-attention.ipynb). Il ne contient pas de récurrence (pas de convolution aussi). Ce qui représente une différence fondamentale comparée aux modèles cités ci-dessus qui utilisent des réseaux récurrents. Cependant il a hérité leur configuration `Encoder - Decoder`. Du fait de l'absence de récurrence, les transformers offrent une meilleure parallélisation donc une réduction du temps d'entraînement.

<img src="images/transformers/transformers.png" width="300">

# Partie Encodage

La partie `encodage` est composée d'une pile de plusieurs encodeurs (6 dans l'article original).

Les encodeurs possèdent la même architecture mais ne partagent pas leurs poids :

Chaque encodeur est formée d'une couche bidirectionnelle de Self-attention suivie d'une couche feed-forward. 

Le premier encodeur reçoit l'embedding de l'input. Pour le reste des encodeurs, la sortie de l'un constitue l'entrée de l'autre. L'attention permet de tenir compte du contexte d’un mot.

<img src="images/transformers/encoder.png" width="200">

# Partie Décodage

 La partie `décodage` est composée d'une pile plusieurs décodeurs (6 dans l'article original).

Les décodeurs possèdent la même architecture mais ne partagent pas leurs poids :
Chaque décodeur est formé :
* Une couche unidirectionnelle de Self-attention   
* Une couche intermédiaire : elle permet au décodeur de se focaliser sur la séquence d'entrée et sa représentation  
* Une couche feed-forward.

La sortie du dernier encodeur est l'entrée de tous les décodeurs.

<img src="images/transformers/decoder.png" width="150">

# Positional encoding

Comme on l'a dit plus haut, les transformers n'utilisent pas de récurrence. Cela peut potentiellement poser des problèmes pour des données séquentielles du fait l'ordre des mots à un sens. Pour remédier à cela, les transformers ajoutent un vecteur à chaque embedding avec le `Positional encoding`. Ces vecteurs ont la même dimension. Le `positional encoding` est utilisé avant les étapes d'encodage et décodage.

<img src="images/transformers/postional_encoding.png">

# Entrainement

Quelques informations relatives à l'entrainement des transformers d'après le papier original :
* Trois types de régularisation
    * Ajout d'une couche de normalisation
    * Utilisation de la technique du Dropout
    * Label Smoothing pour améliorer la précision du modèle
* Utilisation de la méthode d'optimisation `Àdam`


# Prédiction

Le résultat du décodeur est un vecteur contenant des valeurs numériques. Pour passer de la représentation vectorielle à un mot, on utilise une couche linéaire. Celle-ci permet de projeter le vecteur issu du décodeur dans un espace dont la dimension est égale la taille du vocabulaire. Intuitivement, il s'agit donner un score à chaque mot du vocabulaire en se basant le résultat du décodeur. La dernière étape consiste à appliquer une couche `softmax` qui transforme les scores en probabilités. La sortie correspond au mot ayant la probabilité la plus élevée.

<img src="images/transformers/prediction.png">

## Exemple : Traduction d'une phrase

L'objectif est de montrer d'application des transformers. On utilise un modèle déja pré-entrainé pourf de la traduction.

In [1]:
sentence = "I go to the lake"

In [2]:
from transformers import MarianMTModel, MarianTokenizer
import torch

tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

### A. Illustration de l'encodage et du décodage

**Etape 1** : Tokeniser la phrase : convertir en tokens la sequence pour qu'elle soit au format attendu par le modèle

In [3]:
tokens = tokenizer(sentence, return_tensors="pt")
_, nb_tokens = tokens.input_ids.shape
print('Nombre de de tokens = {}'.format(nb_tokens))
for key, value in tokens.items():
    print("{}:\n\t{}".format(key, value))

Nombre de de tokens = 6
input_ids:
	tensor([[   47,   631,    12,     4, 12485,     0]])
attention_mask:
	tensor([[1, 1, 1, 1, 1, 1]])


Tenseur pour stocker les ID des tokens décodés

In [4]:
decoder_input_ids = tokenizer("<pad>", add_special_tokens=False, return_tensors="pt").input_ids

On passe les tokens à l'encodeur et au décodeur

In [5]:
outputs = model(input_ids=tokens.input_ids, 
                decoder_input_ids=decoder_input_ids, 
                return_dict=True)
print("Token wise output: {}".format(outputs.encoder_last_hidden_state.shape))

Token wise output: torch.Size([1, 6, 512])


Les 6 tokens sont encodés avec vecteurs de taille 512.

In [6]:
encoded_sequence = (outputs.encoder_last_hidden_state,)

**Etape 2** : boucle for pour obtenir les logit des tokens décodés ainsi que leur ids

In [7]:
for i in range(nb_tokens):
    lm_logits = model(None, 
                      encoder_outputs=encoded_sequence, 
                      decoder_input_ids=decoder_input_ids,
                      return_dict=True).logits
    next_decoder_input_ids = torch.argmax(lm_logits[:, -1:], axis=-1)
    decoder_input_ids = torch.cat([decoder_input_ids, next_decoder_input_ids], axis=-1)

**Etape 3** : générer le résultats de la traduction avec les ids

In [8]:
print(tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True))

Je vais au lac.


### B. Traduire directement la phrase

In [9]:
# Créer les ids des tokens
input_ids = tokenizer(sentence, return_tensors="pt").input_ids

# Traduction de l'exemple
output_ids = model.generate(input_ids)[0]

# Décodage et affichage des résultats
print(tokenizer.decode(output_ids))

<pad> Je vais au lac.


**Références:**  
[Original papier : Attention Is All You Need](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf)   
[Hugging Face Github ](https://github.com/huggingface/transformers)